# Naver Movie Review Dataset
#### - Model: Doc2vec
#### - POS tagger : Okt
#### - 참고 : 한국어임베딩 교재

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

### 1. Data Load

In [3]:
# train_data = pd.read_table('matched_review_movieid.txt')
df = pd.read_csv('matched_review_movieid.txt', names=['origin_review'], error_bad_lines=False)

In [4]:
df.head()

,origin_review
0,정말 재미있게 본 훌륭한 드라마였다.␞74379
1,졸잼ㅇㄱㅂ니다. 졸잼␞74379
2,CG가 않좋아서 1점입니다␞74379
3,왜 이 드라마의 가치를 몰라보는가... 스토리도 대사도 일부러 만화적으로 만든걸 모...
4,만화 신불사의 1부를 드라마로 옮겼는데


In [5]:
# 데이터셋 전처리 : id와 review 구분
lines = df['origin_review']
test_list = []
review_list = []
id_list = []

for line in lines:
    try:
        tmp = line.split('␞')
        test_list.append(tmp[0])
        tmp[0]
        tmp[1]
    except:
        continue
    review_list.append(tmp[0])
    id_list.append(tmp[1])

In [6]:
data = {'id': id_list, 'review': review_list}
data = pd.DataFrame(data)

In [7]:
data.head()

,id,review
0,74379,정말 재미있게 본 훌륭한 드라마였다.
1,74379,졸잼ㅇㄱㅂ니다. 졸잼
2,74379,CG가 않좋아서 1점입니다
3,74379,왜 이 드라마의 가치를 몰라보는가... 스토리도 대사도 일부러 만화적으로 만든걸 모...
4,74379,진짜 재밌게봤었는데!


### 2. Missing value 처리

In [8]:
# 중복제외 리뷰 갯수 : 636178 -> 612619
data['review'].nunique()

612619

In [9]:
data.drop_duplicates(subset=['review'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
print(len(data))

612619


In [10]:
# 한글, 공백 제외한 문자제거
data['review'] = data['review'].str.replace("[^ㄱ-하-ㅣ가힣]"," ")

In [11]:
data.isnull().values.any()

False

In [12]:
data['review'].replace('',np.nan,inplace=True)
print(data.isnull().sum())

id        0
review    1
dtype: int64


In [13]:
data.loc[data.review.isnull()]

,id,review
400568,90343\n샤룩에 대한 애정만으로 만점,NaN


In [14]:
tmp = data.loc[data.review.isnull()]

In [15]:
missing_value = tmp.loc[400568]['id'].split('\n')

In [16]:
print(missing_value)

['90343', '샤룩에 대한 애정만으로 만점']


In [18]:
# 결측치 제거
data.dropna(axis=0, inplace=True)
print(len(data))

612618


In [19]:
data.append({'id': missing_value[0], 'review': missing_value[1]}, ignore_index=True)

,id,review
0,74379,정말 재미있게 본 륭 드라마였다
1,74379,졸잼ㅇㄱㅂ니다 졸잼
2,74379,가 않좋아서 점입니다
3,74379,왜 이 드라마의 가치를 몰라보는가 스토리도 대사도 일부러 만 적으로 만든걸 모...
4,74379,진짜 재밌게봤었는데
...,...,...
612614,17762,아련 기억과 께 새겨지는 백의 영상
612615,17762,비장하지만 어리석다
612616,17762,예전의기억 저편에 남아 언제나 주인공의 절규가 들리는 영
612617,17762,마지막 주인공이 차를몰면서 절규하며 따라부르는 바하의 토카타와푸가 죽음


In [20]:
print(data.isnull().sum())

id        0
review    0
dtype: int64


### 3. 데이터 전처리 
- 불용어
- 형태소 분석 : Okt(Twitter)

In [27]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.3, train_size=0.7)

In [28]:
print(len(train_data))
print(len(test_data))

428832
183786


In [41]:
train_data.head()

,id,review
86650,19306,무섭더라 꿈에 나올까바 잠도 제대로 못자따
148129,78872,정말 불편하고 안좋은면을 다룬다 그래서 불편하다
334445,129783,아이와 께보니좋으네요
29189,99799,와 마지막에 역대급 소름이돋았다
67786,10397,조선민주주의 인민공 국 장병들을 기리는 영 ㅋㅋ에


- 한국어 불용어(Stopwords)

In [22]:
df_stopwords = pd.read_csv('./korean_stopwords.txt', names=['stopword', 'POS', 'rate'], header=None, delimiter='\t')

In [23]:
df_stopwords.head()

,stopword,POS,rate
0,이,VCP,0.018280
1,있,VA,0.011699
2,하,VV,0.009774
3,것,NNB,0.009733
4,들,XSN,0.006898


In [24]:
stopwords = df_stopwords['stopword'].tolist()

In [25]:
okt = Okt()

#### okt.morphs의 옵션
- norm : normalize의 약자로 정규화
- stem : 각 단어에서 어간 추출

In [26]:
# 텍스트를 형태소 단위로 구분 후 어간 추출
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [30]:
# 불용어처리 X_train 저장
X_train = []
from tqdm import tqdm
for sentence in tqdm(train_data['review']):
    temp_X = []
    temp_X = okt.morphs(sentence, stem = True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

100%|██████████| 428832/428832 [13:11<00:00, 541.62it/s]


In [132]:
# 불용어처리 X_test 저장
X_test = []
for sentence in tqdm(test_data['review']):
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

100%|██████████| 183786/183786 [12:24<00:00, 246.96it/s]


In [33]:
train_data[['review']].head()

,review
86650,무섭더라 꿈에 나올까바 잠도 제대로 못자따
148129,정말 불편하고 안좋은면을 다룬다 그래서 불편하다
334445,아이와 께보니좋으네요
29189,와 마지막에 역대급 소름이돋았다
67786,조선민주주의 인민공 국 장병들을 기리는 영 ㅋㅋ에


In [34]:
print(X_train[:5])

[['무섭다', '꿈', '에', '나오다', '바', '잠도', '제대로', '못자다', '따다'], ['정말', '불편하다', '안좋다', '면', '을', '다루다', '그래서', '불편하다'], ['아이', '와', '께', '보다', '좋다'], ['오다', '마지막', '에', '역대', '급', '소름', '돋다'], ['조선', '민주주의', '인민', '공', '국', '장병', '을', '기리', '는', '영', 'ㅋㅋ', '에']]


In [36]:
len(X_train)

428832

In [42]:
train_data['review_tokens'] = X_train

<ipython-input-42-2f6b0c87bb56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['review_tokens'] = X_train


In [43]:
train_data.head()

,id,review,review_tokens
86650,19306,무섭더라 꿈에 나올까바 잠도 제대로 못자따,"[무섭다, 꿈, 에, 나오다, 바, 잠도, 제대로, 못자다, 따다]"
148129,78872,정말 불편하고 안좋은면을 다룬다 그래서 불편하다,"[정말, 불편하다, 안좋다, 면, 을, 다루다, 그래서, 불편하다]"
334445,129783,아이와 께보니좋으네요,"[아이, 와, 께, 보다, 좋다]"
29189,99799,와 마지막에 역대급 소름이돋았다,"[오다, 마지막, 에, 역대, 급, 소름, 돋다]"
67786,10397,조선민주주의 인민공 국 장병들을 기리는 영 ㅋㅋ에,"[조선, 민주주의, 인민, 공, 국, 장병, 을, 기리, 는, 영, ㅋㅋ, 에]"


### 4. 모델 설계 : Doc2Vec

In [45]:
doc_df = train_data[['id','review_tokens']].values.tolist()
tagged_docs = [TaggedDocument(words=tokens, tags=[movie_id]) for movie_id, tokens in doc_df]

In [46]:
len(tagged_docs)

428832

In [47]:
tagged_docs[0]

TaggedDocument(words=['무섭다', '꿈', '에', '나오다', '바', '잠도', '제대로', '못자다', '따다'], tags=['19306'])

In [48]:
model = Doc2Vec(tagged_docs, dm=1, vector_size=100)

In [131]:
model.docvecs[0]

array([ 0.12862666,  0.19557568,  0.11846325, -0.02422391, -0.34906718,
       -0.20162469, -0.06400381,  0.01379242, -0.26292434,  0.08053754,
        0.02949703,  0.36939222, -0.41916183,  0.2657878 ,  0.16435876,
        0.3640658 ,  0.00557661, -0.10704455, -0.01833933, -0.00849472,
       -0.33401474,  0.14371498,  0.00079793, -0.20409249,  0.10100336,
        0.34552237,  0.0084727 ,  0.27368885, -0.0479448 , -0.3214688 ,
       -0.1300217 ,  0.57871264, -0.47123784,  0.04703402, -0.16656503,
       -0.06848156,  0.28114066,  0.04258383, -0.17462876,  0.24065137,
       -0.22595285,  0.34482336,  0.3525008 ,  0.35502395,  0.02406902,
       -0.3175983 , -0.24082842, -0.11819652, -0.42017007,  0.51992255,
       -0.37836134,  0.14809377,  0.09904487, -0.03935252,  0.5569911 ,
       -0.36290088, -0.2521839 ,  0.32233584,  0.2895356 ,  0.03414615,
        0.4236006 ,  0.06458165, -0.4855738 , -0.1581831 ,  0.12933376,
       -0.28997856, -0.3107875 ,  0.15031959, -0.05903892, -0.04

In [50]:
model.corpus_count

428832

### 5. 모델 평가

In [103]:
import sys, requests, random
from lxml import html
from visualize_utils import visualize_homonym, visualize_between_sentences, \
    visualize_self_attention_scores, visualize_sentences, visualize_words, visualize_between_words

In [106]:
doc2idx = {el:idx for idx, el in enumerate(model.docvecs.doctags.keys())}
use_notebook = True

In [113]:
def most_similar(movie_id, topn=10):
    similar_movies = model.docvecs.most_similar(str(movie_id), topn=topn)
    for movie_id, score in similar_movies:
        print(get_movie_title(movie_id), score)

def get_movie_title(movie_id):
    url = 'http://movie.naver.com/movie/point/af/list.nhn?st=mcode&target=after&sword=%s' % movie_id
    resp = requests.get(url)
    root = html.fromstring(resp.text)
    try:
        title = root.xpath('//div[@class="choice_movie_info"]//h5//a/text()')[0]
    except:
        title = ""
    return title

def get_titles_in_corpus(n_sample=5):
    movie_ids = random.sample(model.docvecs.doctags.keys(), n_sample)
    return {movie_id: get_movie_title(movie_id) for movie_id in movie_ids}

def visualize_movies(n_sample=30, palette="Viridis256", type="between"):
    movie_ids = get_titles_in_corpus(n_sample=n_sample)
    movie_titles = [movie_ids[key] for key in movie_ids.keys()]
    movie_vecs = [model.docvecs[doc2idx[movie_id]] for movie_id in movie_ids.keys()]
    if type == "between":
        visualize_between_words(movie_titles, movie_vecs, palette, use_notebook=use_notebook)
    else:
        visualize_words(movie_titles, movie_vecs, palette, use_notebook=use_notebook)

- 학습 데이터에 포함된 영화 제목 추출

In [125]:
get_titles_in_corpus(5)

{'20106': '난중일기',
 '22787': '자유만세',
 '107959': '탑기어 코리아 시즌4',
 '110366': '싱글 인 LA',
 '122101': '아바타 정글의 비밀'}

- 가장 연관성 높은 영화 추천

In [128]:
most_similar('20106', topn=10)

노 머시 0.8939095139503479
옥토퍼스 0.87286776304245
선택 0.8678715229034424
레아 0.8669946193695068
IT 버블과 같이 잔 여자 0.8667657375335693
스틸 브리딩 0.8663744330406189
패스트 푸드 패스트 우먼 0.8659929633140564
빙점 0.8618744611740112
꽃지 0.8616834282875061
그 여름의 태풍 0.8605736494064331


- 30개 샘플에 대해 시각화

In [109]:
# 히트맵
visualize_movies(type="between")

Loading BokehJS ...

In [115]:
# 2차원 맵
visualize_movies(type="tsne")

Loading BokehJS ...